# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_DFT(lattice, atoms, positions;
                  functionals=LDA(), symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770257750215                   -0.53    9.0    198ms
  2   -2.771681419959       -2.85       -1.31    1.0   94.9ms
  3   -2.771714115608       -4.49       -2.61    1.0    504ms
  4   -2.771714685150       -6.24       -4.04    1.0   93.1ms
  5   -2.771714715152       -7.52       -4.46    2.0    137ms
  6   -2.771714715246      -10.03       -5.39    1.0    133ms
  7   -2.771714715250      -11.46       -6.00    2.0    122ms
  8   -2.771714715250      -12.99       -6.53    1.0    100ms
  9   -2.771714715250      -14.40       -7.77    1.0   97.7ms
 10   -2.771714715250      -14.51       -7.82    2.0    115ms
 11   -2.771714715250      -14.75       -8.50    1.0    102ms


-0.0001345734464263141

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770486464550                   -0.52    9.0    239ms
  2   -2.771777750485       -2.89       -1.33    1.0    150ms
  3   -2.771801406778       -4.63       -2.41    1.0   97.0ms
  4   -2.771801986414       -6.24       -3.11    1.0    101ms
  5   -2.771802074174       -7.06       -4.42    2.0    119ms
  6   -2.771802074350       -9.76       -4.65    1.0    528ms
  7   -2.771802074468       -9.93       -5.17    2.0    113ms
  8   -2.771802074473      -11.29       -5.42    1.0    106ms
  9   -2.771802074476      -11.52       -6.77    1.0    117ms
 10   -2.771802074476      -13.33       -6.94    2.0    119ms
 11   -2.771802074476      -14.88       -7.25    1.0    108ms
 12   -2.771802074476      -14.40       -8.25    1.0    121ms


0.017612221864315332

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.0001345734464263141
Displaced dipole:  0.017612221864315332
Polarizability :   1.7746795310741648


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920963709e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553108540e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852754150172e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694607592728e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088790554955e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.325934533226e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.846841344780e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.846841344780e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.007013834688e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.184366872500e-10
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.623274592871e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.623274592871e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.6232824014667212e-12
└ *  number of opera

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.